In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.float_format', lambda x: '%.5f' % x)

import Config
import datetime
import time
import copy
import BS

from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, ConfusionMatrixDisplay, precision_score

from pymongo import MongoClient
client=MongoClient(Config.DB_Hostname,Config.DB_Port)

def _calc_spread_return(spot, long_start, long_end, short_start, short_end, strike_spread):
    
    spread_benefit = short_start - long_start
    margin = spot*0.02 - spread_benefit
    brokerage = (long_start + long_end + short_start + short_end) * 0.001
    
    if spread_benefit > 0:
        margin += strike_spread
        
    if margin <0:
        print(f"Error_{margin}")
    
    return (long_end - long_start + short_start - short_end - brokerage)/margin


class Gap_Move_Classifier:
    
    def __init__(self, start_date, end_date, prediction_date, underlying, start_holding_period, end_holding_period):
        
        self.start_date = start_date #2017-01-01
        self.end_date = end_date #2017-01-01
        self.prediction_date = prediction_date #2017-01-01
        self.underlying = underlying #NIFTY, BANKNIFTY
        self.underlying_dynamics = None
        self.vol_surface = None
        self.days_to_expiry = None
        self.data_matrix = None
        self.return_matrix = None
        self.price_matrix = None
        self.performance_stats = None
        self.start_holding_period = start_holding_period #in minutes
        self.end_holding_period = end_holding_period #in minutes
        self.strike_range = [-1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1]
        
        #start = time.time()
        self.underlying_dynamics = pd.DataFrame(client[f'{Config.Data_DB}'][f'{self.underlying}OHLC'].find())
        self.underlying_dynamics.drop(columns=['_id'],inplace=True)
        self.underlying_dynamics.sort_values(['date','batch_id'],inplace=True)
        self.underlying_dynamics=self.underlying_dynamics[(self.underlying_dynamics.date>=self.start_date)&(self.underlying_dynamics.date<=self.end_date)]
        self.underlying_dynamics.batch_id=self.underlying_dynamics.batch_id.astype(int)
        self.underlying_dynamics.set_index('date',inplace=True)
        #print(f"Prices Downloaded in {time.time() - start} seconds")

        #start=time.time()
        self.vol_surface=pd.DataFrame(client[f'{Config.Data_DB}']['Vol_Surface'].find({"SYMBOL" : self.underlying}))
        self.vol_surface.drop(columns=['_id', 'CONTRACTS', 'VAL_INLAKH','OPEN_INT', 'CHG_IN_OI', 'SYMBOL'], inplace=True)
        self.vol_surface.sort_values('date',inplace=True)
        self.vol_surface=self.vol_surface[(self.vol_surface.date>=self.start_date)&(self.vol_surface.date<=self.end_date)]
        self.vol_surface.set_index('date',inplace=True)
        #print(f"Vol Surface Downloaded in {time.time() - start} seconds")
        
        #start=time.time()
        self.days_to_expiry = pd.DataFrame(client[f'{Config.Data_DB}']['Days_To_Expiry'].find({"underlying":self.underlying}))
        self.days_to_expiry.drop(columns = ['_id','underlying'],inplace=True)
        self.days_to_expiry.sort_values('date',inplace=True)
        self.days_to_expiry=self.days_to_expiry[(self.days_to_expiry.date>=self.start_date)&(self.days_to_expiry.date<=self.end_date)]
        self.days_to_expiry.set_index('date',inplace=True)
        #print(f"Expiry Dates Downloaded in {time.time()-start} seconds")

        #start=time.time()
        self._make_features()
        #print(f"Features Engineered in {time.time() - start} seconds")

        #start=time.time()
        self._generate_return_matrix()
        #print(f"Features Engineered in {time.time() - start} seconds")
                
    def _make_features(self):
        
        vol_surface = copy.deepcopy(self.vol_surface)
        if self.end_date not in vol_surface.index:
            vol_surface.loc[self.end_date] = [np.nan]*len(vol_surface.columns)

        closing_candle = self.underlying_dynamics[self.underlying_dynamics.batch_id==375 - (self.start_holding_period - 1)][['open']]
        if self.end_date not in closing_candle.index:
            closing_candle.loc[self.end_date] = np.nan
        closing_candle=closing_candle.shift()
        
        opening_candle = self.underlying_dynamics[self.underlying_dynamics.batch_id==1 + (self.end_holding_period - 1)][['close']]
        if self.end_date not in opening_candle.index:
            opening_candle.loc[self.end_date] = np.nan
        
        candle = pd.merge(closing_candle, opening_candle, left_index=True, right_index=True)
        candle.close = np.where(candle.close.isna(), candle.open, candle.close)
        
        self.data_matrix = pd.DataFrame()
        self.data_matrix["gap_move"] = (candle.close - candle.open) / candle.open
        self.data_matrix["gap_move_flag"] = np.where(self.data_matrix.gap_move>=0, 1, 0)
        self.data_matrix = pd.merge(self.data_matrix, vol_surface.shift(), left_index = True, right_index = True).dropna()
        
        candle = pd.merge(candle, self.days_to_expiry, left_index=True, right_index=True)
        candle = pd.merge(candle , 
                          self.days_to_expiry.shift().rename(columns = 
                            {"current_week" : "current_week_yesterday", "next_week": "next_week_yesterday"}), 
                          left_index=True,
                          right_index=True)
        
        candle.current_week_yesterday = np.where(candle.current_week_yesterday == 1, candle.next_week_yesterday, candle.current_week_yesterday)
        candle["IVOL"] = vol_surface["vol_0"].shift()
        candle.dropna(inplace = True)
        
        for strike_range in self.strike_range:
            
            candle[f"{strike_range}_strike_call_and_put"] = candle.open * (1 + strike_range/100) * np.exp(Config.interest_rate * (candle.current_week_yesterday - 1 + self.start_holding_period/1440) / 365)
            
            candle[f"call_{strike_range}_yesterday"] = candle.apply(lambda x : (BS.bs_call(x.open, 
                                                                                x[f"{strike_range}_strike_call_and_put"], 
                                                                                (x.current_week_yesterday - 1 + self.start_holding_period/1440)/ 365, 
                                                                                Config.interest_rate, 
                                                                                x.IVOL)), axis = 1)
            
            candle[f"call_{strike_range}_today"] = candle.apply(lambda x : (BS.bs_call(x.close, 
                                                                                x[f"{strike_range}_strike_call_and_put"], 
                                                                                (x.current_week - 0.75 - self.end_holding_period/1440)/ 365, 
                                                                                Config.interest_rate, 
                                                                                x.IVOL)), axis = 1)

            candle[f"put_{strike_range}_yesterday"] = candle.apply(lambda x : (BS.bs_put(x.open, 
                                                                                x[f"{strike_range}_strike_call_and_put"], 
                                                                                (x.current_week_yesterday - 1 + self.start_holding_period/1440)/ 365, 
                                                                                Config.interest_rate, 
                                                                                x.IVOL)), axis = 1)
            
            candle[f"put_{strike_range}_today"] = candle.apply(lambda x : (BS.bs_put(x.close, 
                                                                                x[f"{strike_range}_strike_call_and_put"], 
                                                                                (x.current_week - 0.75 - self.end_holding_period/1440)/ 365, 
                                                                                Config.interest_rate, 
                                                                                x.IVOL)), axis = 1)
                
        self.price_matrix = candle[(candle.columns[candle.columns.str.contains('call')])| (candle.columns[candle.columns.str.contains('put')]) ]
        self.price_matrix["open"] = candle[['open']]
        self.price_matrix["current_week_yesterday"] = candle[['current_week_yesterday']]
        
    def _generate_return_matrix(self):
        
        self.return_matrix = pd.DataFrame({"date":sorted(self.price_matrix.index)}).set_index('date')
        for index, strike_range_lower in enumerate(self.strike_range[:-1]):
            for strike_range_upper in self.strike_range[index+1:]:

                call_long_yesterday = f"call_{strike_range_lower}_yesterday"
                call_long_today = f"call_{strike_range_lower}_today"
                call_short_yesterday = f"call_{strike_range_upper}_yesterday"
                call_short_today = f"call_{strike_range_upper}_today"
                call_short_strike = f"{strike_range_upper}_strike_call_and_put"
                call_long_strike = f"{strike_range_lower}_strike_call_and_put"
                
                put_long_yesterday = f"put_{strike_range_lower}_yesterday"
                put_long_today = f"put_{strike_range_lower}_today"
                put_short_yesterday = f"put_{strike_range_upper}_yesterday"
                put_short_today = f"put_{strike_range_upper}_today"
                put_short_strike = f"{strike_range_upper}_strike_call_and_put"
                put_long_strike = f"{strike_range_lower}_strike_call_and_put"
                
                if strike_range_lower<=0.75:
                    self.return_matrix[f"call_{strike_range_lower}_{strike_range_upper}"] = self.price_matrix.apply(lambda x: _calc_spread_return(x['open'],
                                                                                        x[call_long_yesterday], 
                                                                                        x[call_long_today], 
                                                                                        x[call_short_yesterday],
                                                                                        x[call_short_today], 
                                                                                        x[call_short_strike] - x[call_long_strike]),
                                                                                        axis=1)
                if strike_range_lower>=-0.75:
                    self.return_matrix[f"put_{strike_range_lower}_{strike_range_upper}"] = self.price_matrix.apply(lambda x: _calc_spread_return(x['open'], 
                                                                                         x[put_long_yesterday], 
                                                                                         x[put_long_today],
                                                                                         x[put_short_yesterday],
                                                                                         x[put_short_today],  
                                                                                         x[put_short_strike] - x[put_long_strike]),
                                                                                         axis=1)
        
    def _generate_performance_stats(self, 
                                    tail = None, 
                                    return_matrix_user = None, 
                                    start_holding_period = None, 
                                    end_holding_period = None):
        
        if start_holding_period is not None:
            self.start_holding_period = start_holding_period
        
        if end_holding_period is not None:
            self.end_holding_period = end_holding_period
            
        if return_matrix_user is not None:
            self.return_matrix = return_matrix_user[return_matrix_user.columns[(return_matrix_user.columns.str.contains('call')) | (return_matrix_user.columns.str.contains('put'))]]

        sim = pd.DataFrame(index = self.return_matrix.columns)
        
        if tail is not None:
            self.return_matrix = self.return_matrix.tail(tail)        
        return_matrix = self.return_matrix.copy()
        
        sim["Sharpe"] = 16 * return_matrix.mean()/return_matrix.std()

        rolling_cumulative = (1 + return_matrix).rolling(60).apply(lambda x : x.prod()).dropna()
        sim["60_Day_Median_Cumulative_Return"] = rolling_cumulative.median()
        sim["60_Day_Mean_Cumulative_Return"] = rolling_cumulative.mean()
        sim["60_Day_Max_Cumulative_Return"] = rolling_cumulative.max()
        sim["60_Day_Min_Cumulative_Return"] = rolling_cumulative.min()

        rolling_cumulative = (1 + return_matrix).rolling(250).apply(lambda x : x.prod()).dropna()
        sim["250_Day_Median_Cumulative_Return"] = rolling_cumulative.median()
        sim["250_Day_Mean_Cumulative_Return"] = rolling_cumulative.mean()
        sim["250_Day_Max_Cumulative_Return"] = rolling_cumulative.max()
        sim["250_Day_Min_Cumulative_Return"] = rolling_cumulative.min()

        sim["Return"] = ((1 + return_matrix).cumprod().tail(1) ** (1/len(return_matrix)) - 1).mean()

        sim["Last_60_Day_Cumulative_Return"] = (1 + return_matrix).tail(60).cumprod().tail(1).mean()
        sim["Last_250_Day_Cumulative_Return"] = (1 + return_matrix).tail(250).cumprod().tail(1).mean()
        sim["MDD"] = (((1 + return_matrix).cumprod() / ((1 + return_matrix).cumprod().cummax())) - 1).min()   
        
        sim["start_holding_period"] = self.start_holding_period
        sim["end_holding_period"] = self.end_holding_period
        
        self.performance_stats = sim.copy()

In [ ]:
start_date = '2015-01-09' #starting data point best left untouchedd
end_date = '2024-02-06'#'2022-01-03'#'2023-07-03' # Last day for which you want prediction - should be next trading day
prediction_date = '2024-01-23' # should be legitimate and after start date and before end date. Only dates >= are processed and pushed into DB
underlying = 'NIFTY' #NIFTY/BANKNIFTY
start_holding_period = 13
end_holding_period = 12
self = Gap_Move_Classifier(start_date, end_date, prediction_date, underlying, start_holding_period, end_holding_period)
self._generate_performance_stats()

In [ ]:
self.return_matrix["call_-1_0.75"]

In [2]:
start_holding_period = list(range(1,16))
end_holding_period = list(range(1,16))
tail = 500
return_summary = []
summary = []
for start in start_holding_period:
    for end in end_holding_period:
        
        print(start,end)
        
        start_date = '2015-01-09' #starting data point best left untouchedd
        end_date = '2024-02-15'#'2022-01-03'#'2023-07-03' # Last day for which you want prediction - should be next trading day
        prediction_date = '2024-01-19' # should be legitimate and after start date and before end date. Only dates >= are processed and pushed into DB
        underlying = 'NIFTY' #NIFTY/BANKNIFTY
        self = Gap_Move_Classifier(start_date, end_date, prediction_date, underlying, start, end)
        self._generate_performance_stats(tail=tail)
        summary.append(self.performance_stats)
        
        temp = pd.merge(self.return_matrix, self.days_to_expiry[['current_week']], left_index = True, right_index = True)
        temp["start_holding_period"] = start
        temp["end_holding_period"] = end
        return_summary.append(temp)
        
summary = pd.concat(summary)
return_summary = pd.concat(return_summary)

1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
2 14
2 15
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
3 13
3 14
3 15
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
4 12
4 13
4 14
4 15
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
5 11
5 12
5 13
5 14
5 15
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
6 10
6 11
6 12
6 13
6 14
6 15
7 1
7 2
7 3
7 4
7 5
7 6
7 7
7 8
7 9
7 10
7 11
7 12
7 13
7 14
7 15
8 1
8 2
8 3
8 4
8 5
8 6
8 7
8 8
8 9
8 10
8 11
8 12
8 13
8 14
8 15
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
9 9
9 10
9 11
9 12
9 13
9 14
9 15
10 1
10 2
10 3
10 4
10 5
10 6
10 7
10 8
10 9
10 10
10 11
10 12
10 13
10 14
10 15
11 1
11 2
11 3
11 4
11 5
11 6
11 7
11 8
11 9
11 10
11 11
11 12
11 13
11 14
11 15
12 1
12 2
12 3
12 4
12 5
12 6
12 7
12 8
12 9
12 10
12 11
12 12
12 13
12 14
12 15
13 1
13 2
13 3
13 4
13 5
13 6
13 7
13 8
13 9
13 10
13 11
13 12
13 13
13 14
13 15
14 1
14 2
14 3
14 4
14 5
14 6
14 7
14 8
14 9
14 10
14 11
14 12
14 13
14 14
14 15
1

In [3]:
return_summary.current_week = np.where(return_summary.current_week == 5, 4, return_summary.current_week)
return_summary.current_week = np.where(return_summary.current_week == 6, 7, return_summary.current_week)
selected_strategies = []
for current_week in sorted(return_summary.current_week.unique()):
    
    print(current_week)
    temp = []
    for start in sorted(return_summary.start_holding_period.unique()):
        for end in sorted(return_summary.end_holding_period.unique()):
            
            self._generate_performance_stats(return_matrix_user = return_summary[(return_summary.current_week == current_week) & (return_summary.start_holding_period == start) & (return_summary.end_holding_period == end)], 
                                   start_holding_period = start, end_holding_period = end)
            temp.append(self.performance_stats)
    
    temp = pd.concat(temp)
    temp["current_week"] = current_week
    selected_strategies.append(temp.sort_values('Return', ascending=False).head(1))
selected_strategies = pd.concat(selected_strategies)
selected_strategies

1
2
3
4
7


,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period,current_week
call_-1_0.25,4.10418,2.89235,3.13043,5.94556,1.54254,NaN,NaN,NaN,NaN,0.02129,1.95793,9.32680,-0.51523,1,4,1
call_-1_0.25,3.13933,2.21037,2.23690,3.17507,1.46273,NaN,NaN,NaN,NaN,0.01186,1.46273,3.36903,-0.34452,9,13,2
call_-1_1,6.00534,6.19020,6.32557,11.76048,2.70207,NaN,NaN,NaN,NaN,0.02614,6.56043,11.90906,-0.43895,9,1,3
call_-1_-0.25,1.63306,1.55440,1.55413,2.40075,0.74708,NaN,NaN,NaN,NaN,0.00366,2.40075,1.41011,-0.33524,9,1,4
call_-1_1,2.32025,1.28665,1.30307,2.05822,0.97318,NaN,NaN,NaN,NaN,0.00828,1.47396,2.29889,-0.45065,9,10,7


In [23]:
selected_return_matrix = []
for strategy, start, end, current_week in zip(selected_strategies.index, 
                                              selected_strategies.start_holding_period, 
                                              selected_strategies.end_holding_period, 
                                              selected_strategies.current_week):
    selected_return_matrix.append(return_summary[(return_summary.start_holding_period == start) & 
                   (return_summary.end_holding_period == end) & 
                   (return_summary.current_week == current_week)][[strategy]].rename(columns = {strategy : "strategy_call_and_put"}))

selected_return_matrix = pd.concat(selected_return_matrix).sort_index()
self._generate_performance_stats(return_matrix_user=selected_return_matrix)
self.performance_stats

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
strategy_call_and_put,3.52255,2.46480,2.71945,6.61197,0.63398,32.64099,39.45580,111.51345,12.24533,0.01430,5.56381,76.06534,-0.53616,15,15


In [24]:
for lag in [250, 500, 750, 1000, 1250, 1500, 1750, 2000, 2228]:
    print(((1+selected_return_matrix.head(lag).tail(250)).cumprod()**(1/250)).tail(1))

            strategy_call_and_put
date                             
2023-02-09                1.01114
            strategy_call_and_put
date                             
2024-02-15                1.01748
            strategy_call_and_put
date                             
2024-02-15                1.01748
            strategy_call_and_put
date                             
2024-02-15                1.01748
            strategy_call_and_put
date                             
2024-02-15                1.01748
            strategy_call_and_put
date                             
2024-02-15                1.01748
            strategy_call_and_put
date                             
2024-02-15                1.01748
            strategy_call_and_put
date                             
2024-02-15                1.01748
            strategy_call_and_put
date                             
2024-02-15                1.01748


In [26]:
selected_return_matrix.describe()

,strategy_call_and_put
count,500.00000
mean,0.01757
std,0.07980
min,-0.28442
25%,-0.02897
50%,0.02547
75%,0.07197
max,0.22955


In [7]:
selected_return_matrix = []
for strategy, start, end, current_week in zip(['call_-1_0.25', 'call_-1_0.75', 'put_-0.75_1', 'call_-1_0',
       'call_-1_0.5'], [1, 8, 1, 9, 13], [9, 14, 14, 1, 13], [1, 2, 3, 4, 7]):
    selected_return_matrix.append(return_summary[(return_summary.start_holding_period == start) & 
                   (return_summary.end_holding_period == end) & 
                   (return_summary.current_week == current_week)][[strategy]].rename(columns = {strategy : "strategy_call_and_put"}))

selected_return_matrix = pd.concat(selected_return_matrix).sort_index()
self._generate_performance_stats(return_matrix_user=selected_return_matrix)
self.performance_stats

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
strategy_call_and_put,2.82342,2.14993,2.48896,6.70429,0.39830,15.68616,20.41021,65.69033,5.67194,0.01128,4.46026,35.35904,-0.62849,15,15


In [8]:
for lag in [250, 500, 750, 1000, 1250, 1500, 1750, 2000, 2228]:
    print(((1+selected_return_matrix.head(lag).tail(250)).cumprod()**(1/250)).tail(1))

            strategy_call_and_put
date                             
2023-02-09                1.00820
            strategy_call_and_put
date                             
2024-02-15                1.01436
            strategy_call_and_put
date                             
2024-02-15                1.01436
            strategy_call_and_put
date                             
2024-02-15                1.01436
            strategy_call_and_put
date                             
2024-02-15                1.01436
            strategy_call_and_put
date                             
2024-02-15                1.01436
            strategy_call_and_put
date                             
2024-02-15                1.01436
            strategy_call_and_put
date                             
2024-02-15                1.01436
            strategy_call_and_put
date                             
2024-02-15                1.01436


In [9]:
temp = summary.copy()
for metric in ['Sharpe', 'MDD', '250_Day_Median_Cumulative_Return', '60_Day_Median_Cumulative_Return',
               'Return', 'Last_250_Day_Cumulative_Return', 'Last_60_Day_Cumulative_Return']:
    temp = temp[temp[metric]>=temp[metric].quantile(0.7)]
    
temp

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
put_-0.75_-0.25,3.47280,1.58902,1.61127,2.59506,0.76533,7.43427,7.48886,12.44982,2.72466,0.00718,2.34681,11.70996,-0.31109,9,4
put_-0.75_-0.25,3.52265,1.61731,1.61814,2.54658,0.76699,7.57549,7.57460,12.52643,2.86074,0.00727,2.33758,11.74885,-0.32397,9,5
put_-0.75_-0.25,3.36263,1.61838,1.57539,2.46144,0.79561,6.20358,6.28946,10.32303,2.99283,0.00695,2.36935,9.60046,-0.31558,9,7
put_-0.75_-0.25,3.54472,1.66505,1.62950,2.51067,0.79523,7.06838,7.14384,11.59863,3.39295,0.00735,2.34854,10.29211,-0.32832,9,9


In [10]:
summary.sort_values("Sharpe", ascending=False).head(5)

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
put_-0.75_-0.5,3.92491,1.33607,1.30650,1.59052,0.90696,3.27391,3.24991,4.30065,2.12609,0.00418,1.50633,3.61512,-0.18210,9,1
put_-0.75_-0.5,3.82399,1.31207,1.29377,1.55558,0.91566,3.12633,3.10137,4.04025,2.15289,0.00406,1.47684,3.34227,-0.16802,1,1
put_-0.75_-0.25,3.79305,1.68610,1.65467,2.55581,0.84739,8.46095,8.58355,14.71023,3.91704,0.00773,2.20575,10.87415,-0.29719,9,1
call_-1_-0.25,3.78529,1.95379,1.89025,3.02187,0.75349,14.73331,15.32585,30.27053,5.43983,0.00955,2.67244,18.97402,-0.40798,9,1
put_-0.75_-0.25,3.71095,1.63702,1.62456,2.44159,0.84395,7.76176,7.84240,12.94468,3.99037,0.00751,2.09828,9.38655,-0.27347,1,1


In [11]:
summary.sort_values("MDD", ascending=False).head(5)

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
put_-0.75_-0.5,3.82399,1.31207,1.29377,1.55558,0.91566,3.12633,3.10137,4.04025,2.15289,0.00406,1.47684,3.34227,-0.16802,1,1
put_-0.75_-0.5,3.37614,1.27845,1.26518,1.58827,0.88019,2.91350,2.83768,3.70180,1.82915,0.00368,1.47907,3.27082,-0.16832,1,2
put_-0.5_-0.25,3.10317,1.27577,1.27384,1.68797,0.89090,2.79568,2.78166,3.72383,1.88434,0.00380,1.49446,3.27841,-0.16909,1,1
put_-0.5_-0.25,2.57705,1.23263,1.22567,1.64232,0.87963,2.37018,2.39146,3.30067,1.70048,0.00316,1.39800,2.62228,-0.17105,3,1
put_-0.5_-0.25,2.57723,1.23679,1.22791,1.65591,0.87131,2.38746,2.40294,3.30849,1.67550,0.00317,1.41230,2.68221,-0.17163,4,1


In [12]:
summary.sort_values("60_Day_Median_Cumulative_Return", ascending=False).head(5)

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
call_-1_0.25,3.01662,2.29801,2.39274,6.02678,0.47124,21.28942,24.62031,64.40936,4.81757,0.01148,5.16737,51.12520,-0.62751,9,10
call_-1_0.5,2.66718,2.24320,2.42808,7.62992,0.39810,15.68338,20.10149,56.71508,3.59457,0.01085,6.00846,48.74940,-0.71000,9,10
call_-1_0.25,2.93581,2.23192,2.29769,5.95582,0.54275,15.09879,18.24779,47.88009,4.81460,0.01103,5.11323,40.55434,-0.57712,9,9
call_-1_0.25,3.10575,2.23188,2.27270,5.28795,0.60054,21.08304,24.22508,68.35844,6.73211,0.01163,4.35097,38.98653,-0.53348,9,1
call_-1_0.25,2.92158,2.21318,2.29041,5.19418,0.51830,17.85515,19.93227,48.74160,5.21183,0.01097,4.55129,36.87248,-0.57765,1,10


In [13]:
summary.sort_values("60_Day_Mean_Cumulative_Return", ascending=False).head(5)

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
call_-1_0.5,2.66718,2.24320,2.42808,7.62992,0.39810,15.68338,20.10149,56.71508,3.59457,0.01085,6.00846,48.74940,-0.71000,9,10
call_-1_0.25,3.01662,2.29801,2.39274,6.02678,0.47124,21.28942,24.62031,64.40936,4.81757,0.01148,5.16737,51.12520,-0.62751,9,10
call_-1_0.5,2.54583,2.06852,2.36661,7.45403,0.46719,12.16425,16.29875,51.32209,3.54642,0.01021,5.62271,36.70670,-0.65130,9,13
call_-1_0.75,2.32702,2.06378,2.35079,8.88044,0.32574,10.05117,13.91575,44.84246,2.23269,0.00943,6.32773,38.14957,-0.78088,9,10
call_-1_0.25,2.87705,2.16920,2.32543,5.99462,0.55050,16.54173,19.87997,56.98759,4.73336,0.01088,4.89127,39.15006,-0.59352,9,13


In [14]:
summary.sort_values("60_Day_Max_Cumulative_Return", ascending=False).head(5)

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
call_-1_1,1.83693,1.70021,2.12933,9.71454,0.28587,3.39243,5.67438,24.77010,0.96590,0.00646,6.12526,19.52229,-0.80774,9,14
call_-1_1,1.92875,1.75233,2.18228,9.69028,0.30860,4.76934,7.60253,27.84906,1.22593,0.00707,5.90802,21.16568,-0.80199,9,13
call_-1_1,2.00739,1.85699,2.20319,9.51018,0.25999,6.04178,9.00716,32.47663,1.23179,0.00759,6.21187,27.11729,-0.83752,9,10
call_-1_1,1.90249,1.75581,2.07893,9.41112,0.31636,3.82644,5.93820,24.70843,1.15833,0.00688,6.00434,19.45393,-0.80087,9,9
call_-1_0.75,2.32702,2.06378,2.35079,8.88044,0.32574,10.05117,13.91575,44.84246,2.23269,0.00943,6.32773,38.14957,-0.78088,9,10


In [15]:
summary.sort_values("60_Day_Min_Cumulative_Return", ascending=False).head(5)

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
put_-0.75_-0.5,3.82399,1.31207,1.29377,1.55558,0.91566,3.12633,3.10137,4.04025,2.15289,0.00406,1.47684,3.34227,-0.16802,1,1
put_-0.75_-0.5,3.92491,1.33607,1.30650,1.59052,0.90696,3.27391,3.24991,4.30065,2.12609,0.00418,1.50633,3.61512,-0.18210,9,1
put_-0.75_-0.5,3.38633,1.28547,1.26034,1.52387,0.90556,2.77964,2.79619,3.70282,1.96624,0.00360,1.40642,2.90486,-0.17213,3,1
put_-0.75_-0.5,3.35410,1.28485,1.26436,1.56784,0.89909,2.83117,2.79744,3.64867,1.87919,0.00364,1.46092,3.13953,-0.18365,15,1
put_-0.75_-0.5,3.40248,1.28725,1.26064,1.52374,0.89658,2.79733,2.79478,3.68205,1.95583,0.00361,1.41891,2.94788,-0.17302,2,1


In [16]:
summary.sort_values("250_Day_Median_Cumulative_Return", ascending=False).head(5)

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
call_-1_0,3.52541,2.17896,2.17417,4.33517,0.69182,22.82001,24.54656,59.05777,6.90851,0.01138,3.56240,35.15518,-0.47686,9,1
call_-1_0,3.35390,2.20980,2.20869,4.43754,0.54270,21.59038,22.79542,52.11592,5.21199,0.01102,4.00022,39.14239,-0.53693,9,10
call_-1_0.25,3.01662,2.29801,2.39274,6.02678,0.47124,21.28942,24.62031,64.40936,4.81757,0.01148,5.16737,51.12520,-0.62751,9,10
call_-1_0.25,3.10575,2.23188,2.27270,5.28795,0.60054,21.08304,24.22508,68.35844,6.73211,0.01163,4.35097,38.98653,-0.53348,9,1
call_-1_0,3.44354,2.08847,2.10684,4.02380,0.68216,19.78351,21.22152,48.47523,7.25060,0.01102,3.24262,27.59238,-0.44490,1,1


In [17]:
summary.sort_values("250_Day_Mean_Cumulative_Return", ascending=False).head(5)

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
call_-1_0.25,3.01662,2.29801,2.39274,6.02678,0.47124,21.28942,24.62031,64.40936,4.81757,0.01148,5.16737,51.12520,-0.62751,9,10
call_-1_0,3.52541,2.17896,2.17417,4.33517,0.69182,22.82001,24.54656,59.05777,6.90851,0.01138,3.56240,35.15518,-0.47686,9,1
call_-1_0.25,3.10575,2.23188,2.27270,5.28795,0.60054,21.08304,24.22508,68.35844,6.73211,0.01163,4.35097,38.98653,-0.53348,9,1
call_-1_0,3.35390,2.20980,2.20869,4.43754,0.54270,21.59038,22.79542,52.11592,5.21199,0.01102,4.00022,39.14239,-0.53693,9,10
call_-1_0,3.44354,2.08847,2.10684,4.02380,0.68216,19.78351,21.22152,48.47523,7.25060,0.01102,3.24262,27.59238,-0.44490,1,1


In [18]:
summary.sort_values("250_Day_Max_Cumulative_Return", ascending=False).head(5)

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
call_-1_0.25,3.10575,2.23188,2.27270,5.28795,0.60054,21.08304,24.22508,68.35844,6.73211,0.01163,4.35097,38.98653,-0.53348,9,1
call_-1_0.25,3.01662,2.29801,2.39274,6.02678,0.47124,21.28942,24.62031,64.40936,4.81757,0.01148,5.16737,51.12520,-0.62751,9,10
call_-1_0,3.52541,2.17896,2.17417,4.33517,0.69182,22.82001,24.54656,59.05777,6.90851,0.01138,3.56240,35.15518,-0.47686,9,1
call_-1_0.5,2.69313,2.08300,2.22999,5.57104,0.50589,14.58316,18.09119,57.66101,5.34125,0.01076,4.74524,31.02888,-0.58250,9,1
call_-1_0.25,2.87705,2.16920,2.32543,5.99462,0.55050,16.54173,19.87997,56.98759,4.73336,0.01088,4.89127,39.15006,-0.59352,9,13


In [19]:
summary.sort_values("250_Day_Min_Cumulative_Return", ascending=False).head(5)

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
call_-1_0,3.44354,2.08847,2.10684,4.02380,0.68216,19.78351,21.22152,48.47523,7.25060,0.01102,3.24262,27.59238,-0.44490,1,1
call_-1_0.25,3.01848,2.08501,2.18208,4.81770,0.59379,18.04126,20.17443,53.52614,7.17726,0.01116,3.78599,28.24647,-0.50993,1,1
call_-1_0,3.52541,2.17896,2.17417,4.33517,0.69182,22.82001,24.54656,59.05777,6.90851,0.01138,3.56240,35.15518,-0.47686,9,1
call_-1_0.25,3.10575,2.23188,2.27270,5.28795,0.60054,21.08304,24.22508,68.35844,6.73211,0.01163,4.35097,38.98653,-0.53348,9,1
call_-1_-0.25,3.68725,1.86694,1.84108,2.86310,0.76115,13.11534,13.59326,25.88083,5.62593,0.00924,2.52090,15.66543,-0.38100,1,1


In [20]:
summary.sort_values("Return", ascending=False).head(5)

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
call_-1_0.25,3.10575,2.23188,2.27270,5.28795,0.60054,21.08304,24.22508,68.35844,6.73211,0.01163,4.35097,38.98653,-0.53348,9,1
call_-1_0.25,3.01662,2.29801,2.39274,6.02678,0.47124,21.28942,24.62031,64.40936,4.81757,0.01148,5.16737,51.12520,-0.62751,9,10
call_-1_0,3.52541,2.17896,2.17417,4.33517,0.69182,22.82001,24.54656,59.05777,6.90851,0.01138,3.56240,35.15518,-0.47686,9,1
call_-1_0.25,3.01848,2.08501,2.18208,4.81770,0.59379,18.04126,20.17443,53.52614,7.17726,0.01116,3.78599,28.24647,-0.50993,1,1
call_-1_0.25,2.93581,2.23192,2.29769,5.95582,0.54275,15.09879,18.24779,47.88009,4.81460,0.01103,5.11323,40.55434,-0.57712,9,9


In [21]:
summary.sort_values("Last_60_Day_Cumulative_Return", ascending=False).head(5)

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
call_-1_0.75,2.32702,2.06378,2.35079,8.88044,0.32574,10.05117,13.91575,44.84246,2.23269,0.00943,6.32773,38.14957,-0.78088,9,10
call_-1_1,2.00739,1.85699,2.20319,9.51018,0.25999,6.04178,9.00716,32.47663,1.23179,0.00759,6.21187,27.11729,-0.83752,9,10
call_-1_0.75,2.13778,1.87290,2.25396,8.84689,0.35905,5.71706,8.74371,33.24608,1.79236,0.00828,6.15870,26.85233,-0.74077,9,14
call_-1_0.75,2.22768,1.94693,2.22811,8.76180,0.39073,6.59530,9.44177,34.75044,2.14121,0.00878,6.14533,28.01050,-0.73458,9,9
call_-1_1,1.83693,1.70021,2.12933,9.71454,0.28587,3.39243,5.67438,24.77010,0.96590,0.00646,6.12526,19.52229,-0.80774,9,14


In [22]:
summary.sort_values("Last_250_Day_Cumulative_Return", ascending=False).head(5)

,Sharpe,60_Day_Median_Cumulative_Return,60_Day_Mean_Cumulative_Return,60_Day_Max_Cumulative_Return,60_Day_Min_Cumulative_Return,250_Day_Median_Cumulative_Return,250_Day_Mean_Cumulative_Return,250_Day_Max_Cumulative_Return,250_Day_Min_Cumulative_Return,Return,Last_60_Day_Cumulative_Return,Last_250_Day_Cumulative_Return,MDD,start_holding_period,end_holding_period
call_-1_0.25,3.01662,2.29801,2.39274,6.02678,0.47124,21.28942,24.62031,64.40936,4.81757,0.01148,5.16737,51.12520,-0.62751,9,10
call_-1_0.25,2.86480,2.03203,2.22128,5.66774,0.50808,17.23055,20.64364,56.68293,3.10403,0.01066,4.86345,50.96439,-0.61953,9,4
call_-1_0.25,2.88826,2.10588,2.21540,5.53114,0.51199,17.43514,20.25565,56.67652,3.38123,0.01076,4.87864,50.44394,-0.60704,9,5
call_-1_0.5,2.66718,2.24320,2.42808,7.62992,0.39810,15.68338,20.10149,56.71508,3.59457,0.01085,6.00846,48.74940,-0.71000,9,10
call_-1_0.5,2.48866,1.95677,2.18900,6.87907,0.42089,12.27964,15.59492,50.15169,2.18245,0.00976,5.37492,44.61971,-0.69926,9,4


In [ ]:
# summary.to_csv('Nifty_Full.csv')

In [ ]:
return_summary.to_csv('Nifty_Full_Sim.csv')

In [ ]:
(1 + self.return_matrix["call_-1_0.75"]).rolling(250).apply(lambda x: x.prod()).dropna().quantile(0.0025)

In [ ]:
27357.49229**(1/1000)

In [ ]:
import numpy as np
future = 21393
future*=np.exp(0.07*14/248)

In [ ]:
future*0.99

In [ ]:
future*1.005